In [13]:
import pandas as pd
import numpy as np
all=pd.read_csv('股票列表.csv',converters={'symbol':str})
all=all[all['list_date']<=20180101]
all=all['symbol'].tolist()

In [14]:
import os
corr=[]
for i in all:
    temp_data=pd.read_csv('日线数据/'+i+'.csv')[['ts_code','trade_date','close']]
    temp_data=temp_data[temp_data['trade_date']>=20180101]
    temp_data=temp_data[temp_data['trade_date']<=20210301]
    if len(temp_data['close'].tolist())<=500:
        continue
    temp_data=temp_data['close'].tolist()[-500:]
    corr.append(temp_data)

第一二步都是在筛选数据

In [15]:
corr=np.array(corr)
print(np.corrcoef(corr))
pearson_corr=np.corrcoef(corr)

[[ 1.          0.47942816  0.46197647 ... -0.20094313  0.34572632
   0.3319213 ]
 [ 0.47942816  1.          0.47910447 ...  0.39481457  0.67481181
   0.69891255]
 [ 0.46197647  0.47910447  1.         ...  0.03837329  0.18889528
   0.66944931]
 ...
 [-0.20094313  0.39481457  0.03837329 ...  1.          0.25240146
   0.40631008]
 [ 0.34572632  0.67481181  0.18889528 ...  0.25240146  1.
   0.35799015]
 [ 0.3319213   0.69891255  0.66944931 ...  0.40631008  0.35799015
   1.        ]]


第三步得到相关矩阵

In [16]:
model_data=[]
useful_names=[]
for i in all:
    temp_data=pd.read_csv('日线数据/'+i+'.csv')[['ts_code','trade_date','pct_chg']]
    temp_data=temp_data[temp_data['trade_date']>=20180101]
    temp_data=temp_data[temp_data['trade_date']<=20210301]
    if len(temp_data['pct_chg'].tolist())<=500:
        continue
    temp_data=temp_data['pct_chg'].tolist()[-500:]
    model_data.append(temp_data)
    useful_names.append(i)
model_data=np.array(model_data)

构建数据矩阵

In [17]:
def find_most_important(temp_feature_improtance):
    temp_feature_improtance=np.array(temp_feature_improtance)
    connection=[0]*len(temp_feature_improtance)
    temp_index=[]
    konw_max=[]
    percent=0
    while 1:
        temp_max=max(temp_feature_improtance)
        if temp_max<=0.01:
            break
        index=np.argmax(temp_feature_improtance)
        temp_index.append(index)
        konw_max.append(temp_max)
        percent=percent+temp_feature_improtance[index]
        temp_feature_improtance[index]=temp_max-10
        if percent>=0.85:
            break
    if len(temp_index)!=0:
        for i in range(len(temp_index)):
            index=temp_index[i]
            connection[index]=konw_max[i]
    return connection

In [18]:
import xgboost
from numpy import loadtxt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
decide_matrix=(pearson_corr>=0.9)&(pearson_corr!=1)
connect_matrix=[]
for i in range(model_data.shape[0]):
    copy=decide_matrix[i]
    result=[0]*len(decide_matrix[i])
    label=model_data[i]
    location=np.where(copy==True)
    feature=model_data[location[0]].T
    if feature.shape[1]==0:
        connect_matrix.append(result)
        continue
    seed = 7
    test_size = 0.01
    X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size=test_size, random_state=seed)
    model = XGBRegressor()
    model.fit(X_train, y_train)
    feature_improtace=model.feature_importances_
    feature_improtace=find_most_important(feature_improtace)
    location=location[0]
    for m in range(len(location)):
        result[location[m]]=feature_improtace[m]
    result[i]=0
    connect_matrix.append(result)
connect_matrix=np.array(connect_matrix)

<font color=red>将相关矩阵转化为feature importance矩阵</font>

In [19]:
print(connect_matrix)
np.count_nonzero(connect_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


25503

In [20]:
print(connect_matrix)
np.sum(connect_matrix!=0)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


25503

In [21]:
print(model_data.shape)
print(len(useful_names))
print(useful_names[0])

(3412, 500)
3412
000001


In [22]:
# np.save('connect_matrix.npy',connect_matrix)

In [23]:
import pandas as pd
up_and_down=pd.read_csv('up_and_down.csv')
risk_score=np.zeros((3412,3412))
for i in range(up_and_down.shape[0]):
    temp_code=str(up_and_down.iloc[i,1][0:6])
    temp_date=up_and_down.iloc[i,0]
    big_change=up_and_down.iloc[i,2]
    if temp_code in useful_names:
        temp_index=useful_names.index(temp_code)
        temp_connection=connect_matrix[temp_index]
        temp_location=np.where(temp_connection!=0)
        temp_location=temp_location[0]
        if len(temp_location)==0:
            continue
        for m in temp_location:
            temp_rixian=pd.read_csv('日线数据/'+str(useful_names[m])+'.csv')
            temp_rixian=temp_rixian[temp_rixian['trade_date']==temp_date]
            if temp_rixian.shape[0]==0:
                continue
            temp_change=temp_rixian.iloc[0,8]
            if temp_change*big_change>0:
                risk_score[temp_index,m]=1*temp_connection[m]

这里是仿真过程

In [24]:
np.save('risk.npy',risk_score)

In [25]:
risk

NameError: name 'risk' is not defined